In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from math import *
from numpy import *
from sklearn.pipeline import Pipeline
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
import os
import string
import gensim
from gensim.models import Word2Vec
import pandas as pd
import random
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('./train_pe.csv')
data

,Identifier,TestName,Result,Pleural_effusion
0,2,X-Ray Chest PA/AP View,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,0
1,3,CT Thorax / Chest - HRCT Plain,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,0
2,4,X-ray (any Portable single exposure),{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,0
3,5,X-Ray Chest PA/AP View,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,0
4,6,X-Ray Chest PA/AP View,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,0
5,7,X-Ray Chest PA/AP View,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,0
6,8,X-Ray Chest PA/AP View,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,0
7,9,USG Whole Abdomen,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,1
8,10,X-Ray Chest PA/AP View,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,0
9,11,X-Ray Chest PA/AP View,{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fon...,0


In [3]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
data = data.Result[0]
print ("\n-----\n".join(tokenizer.tokenize(data)))

{\rtf1\ansi\ansicpg1252\deff0\deflang1033{\fonttbl{\f0\froman\fprq2\fcharset0 Times New Roman;}{\f1\fnil\fcharset0 Microsoft Sans Serif;}}
{\colortbl ;\red0\green0\blue0;}
{\stylesheet{ Normal;}{\s1 heading 1;}{\s2 heading 2;}}
\viewkind4\uc1\pard\keepn\s2\cf1\b\f0\fs21 Investigation: X-Ray - Chest (Portable) \par
\par
\pard Results:\b0\par
\par
Heterogeneous opacity seen in left upper and mid zones with shifting of trachea towards left and left hila superiorly - s/o left upper lobe collapse.
-----
\par
\par
Right lung appears unremarkable.
-----
\par
\par
CP angles and domes of the diaphragm are normal.\par
\par
Cardiac size and configuration is normal.\par
\par
Please correlate clinically.
-----
\par
\cf0\f1\fs17\par
}


In [4]:
data.size

AttributeError: 'str' object has no attribute 'size'

In [5]:
data.shape

AttributeError: 'str' object has no attribute 'shape'

In [6]:
data.Result[0]

AttributeError: 'str' object has no attribute 'Result'

In [7]:
import re
def striprtf(text):
   pattern = re.compile(r"\\([a-z]{1,32})(-?\d{1,10})?[ ]?|\\'([0-9a-f]{2})|\\([^a-z])|([{}])|[\r\n]+|(.)", re.I)
   # control words which specify a "destionation".
   destinations = frozenset((
      'aftncn','aftnsep','aftnsepc','annotation','atnauthor','atndate','atnicn','atnid',
      'atnparent','atnref','atntime','atrfend','atrfstart','author','background',
      'bkmkend','bkmkstart','blipuid','buptim','category','colorschememapping',
      'colortbl','comment','company','creatim','datafield','datastore','defchp','defpap',
      'do','doccomm','docvar','dptxbxtext','ebcend','ebcstart','factoidname','falt',
      'fchars','ffdeftext','ffentrymcr','ffexitmcr','ffformat','ffhelptext','ffl',
      'ffname','ffstattext','field','file','filetbl','fldinst','fldrslt','fldtype',
      'fname','fontemb','fontfile','fonttbl','footer','footerf','footerl','footerr',
      'footnote','formfield','ftncn','ftnsep','ftnsepc','g','generator','gridtbl',
      'header','headerf','headerl','headerr','hl','hlfr','hlinkbase','hlloc','hlsrc',
      'hsv','htmltag','info','keycode','keywords','latentstyles','lchars','levelnumbers',
      'leveltext','lfolevel','linkval','list','listlevel','listname','listoverride',
      'listoverridetable','listpicture','liststylename','listtable','listtext',
      'lsdlockedexcept','macc','maccPr','mailmerge','maln','malnScr','manager','margPr',
      'mbar','mbarPr','mbaseJc','mbegChr','mborderBox','mborderBoxPr','mbox','mboxPr',
      'mchr','mcount','mctrlPr','md','mdeg','mdegHide','mden','mdiff','mdPr','me',
      'mendChr','meqArr','meqArrPr','mf','mfName','mfPr','mfunc','mfuncPr','mgroupChr',
      'mgroupChrPr','mgrow','mhideBot','mhideLeft','mhideRight','mhideTop','mhtmltag',
      'mlim','mlimloc','mlimlow','mlimlowPr','mlimupp','mlimuppPr','mm','mmaddfieldname',
      'mmath','mmathPict','mmathPr','mmaxdist','mmc','mmcJc','mmconnectstr',
      'mmconnectstrdata','mmcPr','mmcs','mmdatasource','mmheadersource','mmmailsubject',
      'mmodso','mmodsofilter','mmodsofldmpdata','mmodsomappedname','mmodsoname',
      'mmodsorecipdata','mmodsosort','mmodsosrc','mmodsotable','mmodsoudl',
      'mmodsoudldata','mmodsouniquetag','mmPr','mmquery','mmr','mnary','mnaryPr',
      'mnoBreak','mnum','mobjDist','moMath','moMathPara','moMathParaPr','mopEmu',
      'mphant','mphantPr','mplcHide','mpos','mr','mrad','mradPr','mrPr','msepChr',
      'mshow','mshp','msPre','msPrePr','msSub','msSubPr','msSubSup','msSubSupPr','msSup',
      'msSupPr','mstrikeBLTR','mstrikeH','mstrikeTLBR','mstrikeV','msub','msubHide',
      'msup','msupHide','mtransp','mtype','mvertJc','mvfmf','mvfml','mvtof','mvtol',
      'mzeroAsc','mzeroDesc','mzeroWid','nesttableprops','nextfile','nonesttables',
      'objalias','objclass','objdata','object','objname','objsect','objtime','oldcprops',
      'oldpprops','oldsprops','oldtprops','oleclsid','operator','panose','password',
      'passwordhash','pgp','pgptbl','picprop','pict','pn','pnseclvl','pntext','pntxta',
      'pntxtb','printim','private','propname','protend','protstart','protusertbl','pxe',
      'result','revtbl','revtim','rsidtbl','rxe','shp','shpgrp','shpinst',
      'shppict','shprslt','shptxt','sn','sp','staticval','stylesheet','subject','sv',
      'svb','tc','template','themedata','title','txe','ud','upr','userprops',
      'wgrffmtfilter','windowcaption','writereservation','writereservhash','xe','xform',
      'xmlattrname','xmlattrvalue','xmlclose','xmlname','xmlnstbl',
      'xmlopen',
   ))
   # Translation of some special characters.
   specialchars = {
      'par': '\n',
      'sect': '\n\n',
      'page': '\n\n',
      'line': '\n',
      'tab': '\t',
      'emdash': u'\u2014',
      'endash': u'\u2013',
      'emspace': u'\u2003',
      'enspace': u'\u2002',
      'qmspace': u'\u2005',
      'bullet': u'\u2022',
      'lquote': u'\u2018',
      'rquote': u'\u2019',
      'ldblquote': u'\201C',
      'rdblquote': u'\u201D', 
   }
   stack = []
   ignorable = False       # Whether this group (and all inside it) are "ignorable".
   ucskip = 1              # Number of ASCII characters to skip after a unicode character.
   curskip = 0             # Number of ASCII characters left to skip
   out = []                # Output buffer.
   for match in pattern.finditer(text):
      word,arg,hex,char,brace,tchar = match.groups()
      if brace:
         curskip = 0
         if brace == '{':
            # Push state
            stack.append((ucskip,ignorable))
         elif brace == '}':
            # Pop state
            ucskip,ignorable = stack.pop()
      elif char: # \x (not a letter)
         curskip = 0
         if char == '~':
            if not ignorable:
                out.append(u'\xA0')
         elif char in '{}\\':
            if not ignorable:
               out.append(char)
         elif char == '*':
            ignorable = True
      elif word: # \foo
         curskip = 0
         if word in destinations:
            ignorable = True
         elif ignorable:
            pass
         elif word in specialchars:
            out.append(specialchars[word])
         elif word == 'uc':
            ucskip = int(arg)
         elif word == 'u':
            c = int(arg)
            if c < 0: c += 0x10000
            if c > 127: out.append(chr(c))
            else: out.append(chr(c))
            curskip = ucskip
      elif hex: # \'xx
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            c = int(hex,16)
            print(c)
            if c > 127: out.append(chr(c))
            else: out.append(chr(c))
      elif tchar:
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            out.append(tchar)
   return ''.join(out)

In [8]:
df = pd.read_csv("train_pe.csv", usecols=[2])

In [9]:
for i,j in df.iterrows(): 
    text=df.Result[i]
    df.Result[i]=striprtf(text)
for i,j in df.iterrows(): 
    text=df.Result[i]
    df.Result[i]=text.lower()
to_remove = ['no', 'nor']
new_stopwords = set(stopwords.words('english')).difference(to_remove)
for i,j in df.iterrows():                  
    text=df.Result[i]
    words=text.split()
    meaningful_words = [w for w in words if not w in new_stopwords]   
    df.Result[i]=(" ".join( meaningful_words))
df

183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
183
216
216
216
216
216
216
216
183
183
183
183
183
183
216
216
216
216
216
216
216
216
216
216
183
183
183
183
183
183
183
216
216
216
216
216
216
216
216
216
216
216
216
216
183
183
183
183
183


,Result
0,investigation: x-ray - chest (portable) result...
1,hrct chest (non contrast) spiral scanning thor...
2,chest x-ray pa view no significant interval ch...
3,investigation: x-ray - chest ap (portable) vie...
4,x-ray chest pa/ap view 26-aug-2017: results: r...
5,x-ray chest pa/ap view 28-aug-2017: results: r...
6,x-ray chest pa/ap view 22-aug-2017: results: c...
7,usg whole abdomen 22-aug-2017: liver: normal s...
8,x-ray chest pa/ap view 23-aug-2017: results: c...
9,x-ray chest pa/ap view 21-aug-2017: results: n...


In [10]:
workabledata = pd.concat([data, df], axis=1)

TypeError: cannot concatenate object of type "<class 'str'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

'{\\rtf1\\ansi\\ansicpg1252\\deff0\\deflang1033{\\fonttbl{\\f0\\froman\\fprq2\\fcharset0 Times New Roman;}{\\f1\\fnil\\fcharset0 Microsoft Sans Serif;}}\r\n{\\colortbl ;\\red0\\green0\\blue0;}\r\n{\\stylesheet{ Normal;}{\\s1 heading 1;}{\\s2 heading 2;}}\r\n\\viewkind4\\uc1\\pard\\keepn\\s2\\cf1\\b\\f0\\fs21 Investigation: X-Ray - Chest (Portable) \\par\r\n\\par\r\n\\pard Results:\\b0\\par\r\n\\par\r\nHeterogeneous opacity seen in left upper and mid zones with shifting of trachea towards left and left hila superiorly - s/o left upper lobe collapse. \\par\r\n\\par\r\nRight lung appears unremarkable. \\par\r\n\\par\r\nCP angles and domes of the diaphragm are normal.\\par\r\n\\par\r\nCardiac size and configuration is normal.\\par\r\n\\par\r\nPlease correlate clinically. \\par\r\n\\cf0\\f1\\fs17\\par\r\n}\r\n'

In [12]:
workabledata = pd.concat([data, df], axis=1)

TypeError: cannot concatenate object of type "<class 'str'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [13]:
workabledata.Result[0]

NameError: name 'workabledata' is not defined

In [ ]:
df.Result[1]

In [ ]:
workabledata.groupby(['TestName']).count().sort_values(['Identifier'])

In [ ]:
 workabledata.to_csv("workablehopefuk.csv", index=False, encoding='utf8')

In [ ]:
workabledata.to_excel("workablehopefukes.xlsx", sheet_name="Sheet 1", index=False)

In [ ]:
workabledata

In [ ]:
test_related_pe=['CT Thorax / Chest - HRCT Contrast','CT Thorax / Chest - HRCT Plain','CT Thorax / Chest - Plain','CT Whole Abdomen - Plain','CT Whole Abdomen with Contrast','CT Whole Thorax (Contrast)','CT Whole Thorax (Plain)','MRI Cholangiography (MRCP)','Ultrasound Chest','Ultrasound KUB','Ultrasound Whole Abdomen','USG Chest','USG Upper Abdomen','USG Whole Abdomen','X-ray (any Portable single exposure)','X-Ray Chest PA/AP View']

In [ ]:
import re
unique_words=set()
punctuations = list(string.punctuation)
for i,j in workabledata.iterrows():
    test=workabledata.TestName[i]
    text=workabledata.Result[i]
    if(test in test_related_pe ):
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~+/='''
        text3=""
        for char in text:
            if char not in punctuations:
                text3 = text3 + char
        unique_words_temp=set(text3.split())
        unique_words=unique_words.union(unique_words_temp)
        
        
#text=workabledata.Result[503]
#punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~+/='''
#text3=""
#for char in text:
#    if char not in punctuations:
#        text3 = text3 + char
#text3
#len(unique_words)
unique_words_list=[]
for e in unique_words:
    #print(e)
    unique_words_list.append(e)
unique_words_list
def oopsie(x):
    flag=False
    for i in x:
        if(i.isdigit()):
            flag=True
            break
    return flag
li=[x for x in unique_words_list if not oopsie(x)]
sorted(li)
len(li)
for x in li:
    if x in new_stopwords:
        li.remove(x)
len(li)
li.remove('kk')
li
workabledata

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
new_stopwords = set(stopwords.words('english')).difference(to_remove)
vectorizer = CountVectorizer(lowercase=True, stop_words=new_stopwords)
matrix = vectorizer.fit_transform(workabledata.Result)

In [ ]:
s=workabledata.copy()
sd=workabledata[workabledata.Pleural_effusion>0]
sdt=sd.copy()
ops=1
sd = sd.reset_index(drop=True)
sd

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from collections import Counter

top_N = 20

txt = sd.Result.str.lower().str.replace(r'\|', ' ').str.cat(sep=' ')
#print(txt)

words = nltk.tokenize.word_tokenize(txt)

word_dist = nltk.FreqDist(words)

words_except_stop_dist = nltk.FreqDist(w for w in words if (w not in new_stopwords and w not in string.punctuation and w!='.') ) 

#print('All frequencies, including STOPWORDS:')
#print('=' * 60)
#rslt = pd.DataFrame(word_dist.most_common(top_N),
#                    columns=['Word', 'Frequency'])
#print(rslt)
#print('=' * 60)

rslt = pd.DataFrame(words_except_stop_dist.most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')

matplotlib.style.use('ggplot')

#rslt.plot.bar(rot=0)
word_counter = Counter(words_except_stop_dist)
words
oofsd=Counter(nltk.bigrams(words))
#print(oofsd)
oofsd.most_common()
oofsdf=[]
for x in oofsd:
    txt=x
    if(txt[0]=='no'):
        #print(txt)
        oofsdf.append(txt)
oofsdf

In [ ]:
twword=[('mid','zones'),('pleural', 'effusion'),('right', 'lobe'),('lower', 'lobe'),('middle', 'lobe'),('upper', 'lobe'),('left', 'lobe'),('right', 'side'),('left', 'side'),('left', 'kidney'),('right', 'kidney'),('wall', 'thickening'),('bilateral', 'kidneys'),('focal', 'lesion'),('urinary', 'bladder'),('gall', 'bladder'),('mass', 'seen'),('wall', 'oedema'),('lymph', 'nodes'),('thick', 'walled'),('free', 'fluid'),('soft', 'tissues'),('bladder', 'wall')]
thrword=[('left', 'upper_lobe'),('right', 'upper_lobe'),('right','pleural_effusion'),('gall_bladder', 'walls'),('left', 'pleural_effusion')]
to2=[]
s
for i,j in s.iterrows():
    text=s.Result[i]
    to=nltk.tokenize.word_tokenize(text)
    oof=[]
    for x in range(len(to)-1):
        a=(to[x],to[x+1])
        if(a in twword):
            oof.append(x)
    for x in oof:
        a=to[x]+'_'+to[x+1]
        to[x]=a
        to[x+1]=''

    oof2=[]
    for x in range(len(to)-1):
        a=(to[x],to[x+1])
        if(a in thrword):
            #print(a)
            oof2.append(x)
            #print(a)
        #print(len(oof2))
    for x in oof2:
        a=to[x]+'_'+to[x+1]
        to[x]=a
        to[x+1]=''
    oof3=[]
    for x in range(len(to)-1):
        a=(to[x],to[x+1])
        if(a in oofsdf):
            #print(a)
            oof3.append(x)
            #print(a)
        #print(len(oof3))
    for x in oof3:
        a=to[x]+'_'+to[x+1]
        to[x]=a
        to[x+1]=''
    #print(to)
    #to2=to2+to
    #print(len(to))
    for x in to:
        if(x==''):
            to.remove(x)
    txt=' '.join(to)
    df.Result[i]=txt
    
#s=pd.concat([s,df],axis=1)
#workabledata = pd.concat([data, df], axis=1)
#workabledata.Result[]
    #print(txt)
#len(to2)

#xd=Counter(to2)
#xd.most_common()
    

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 
lemmatizer = WordNetLemmatizer() 
for i,j in s.iterrows():
    text=s.Result[i]
    to=nltk.tokenize.word_tokenize(text)
    #print(to)
    for x in range (len(to)):
        y=to[x]
        to[x]=lemmatizer.lemmatize(y)
        to[x]=ps.stem(to[x])
        
    #print(to)
        #print(x)
    txt=' '.join(to)
    print(txt)
    df.Result[i]=txt
    #print(to)

    

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
new_stopwords = set(stopwords.words('english')).difference(to_remove)
matrix = vectorizer.fit_transform(to2)
matrix=matrix.toarray()
#sorted_word_counts = sorted(list(word_counter.values()), reverse=True)
#plt.loglog(sorted_word_counts)
#plt.ylabel("Freq")
#plt.xlabel("Word Rank");

In [ ]:
#s=pd.concat([s,df],axis=1)
#del s['Result']
#s=pd.concat([s,df],axis=1)
s

In [ ]:
#plt.hist(sorted_word_counts, bins=1623, log=True);
import gensim
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
lda=LDA(n_components=2)

In [ ]:
s

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
new_stopwords = set(stopwords.words('english')).difference(to_remove)
vectorizer = CountVectorizer(lowercase=True, stop_words=new_stopwords)
#matrix = vectorizer.fit_transform(to2)
#matrix=matrix.toarray()
matrix = vectorizer.fit_transform(s.Result)
m2=vectorizer.fit_transform(text.split())
m2t=m2.toarray()
print(m2t)
m2t.resize(matrix.shape)
matrix=matrix.toarray()
print(matrix)
matrix
#print(len(matrix))
x=matrix
print(m2t.shape)
print(x.shape)
y=s.Pleural_effusion
yt=y.array
x.shape
myList = list(nltk.bigrams(words))
for i in myList:
    if i[0]=='normal':
        myList.remove(i)
for i in myList:
    if i[1]=='normal':
        myList.remove(i) 
myList2no=[]
for i in myList:
    if i[0]=='no':
        myList2no.append(i)
        myList.remove(i)
myListnod=Counter(myList2no)
myListnos=myListnod.most_common()
#for i in myListdfg:
    #if(i[1]==1):
        #myList.remove(i[0])
myListsd=Counter(myList)
myListdfg=myListsd.most_common()
(myListdfg)
print(len(myListdfg))
print(len(myListnos))
(myList)
twword=[('right', 'lobe'),('left', 'lobe'),('right', 'side'),('left', 'side'),('mass', 'seen'),('wall', 'oedema'),('lymph', 'nodes'),('thick', 'walled'),('free', 'fluid'),('soft', 'tissues'),('bladder', 'wall')]
thrword=[('left', 'upper','lobe'),('right', 'upper','lobe'),('left', 'kidney'),('right', 'kidney'),('wall', 'thickening'),('bilateral', 'kidneys'),('focal', 'lesion'),('urinary', 'bladder'),('gall', 'bladder'),('pleural', 'effusion'),('right', 'pleural', 'effusion'),('gall', 'bladder', 'walls'),('left', 'pleural', 'effusion')]


In [ ]:
word_counter_list=[word_counter]

In [ ]:
bagsofwords = [Counter(re.findall(r'\w+', txt))for txt in workabledata.Result]

In [ ]:
from nltk.classify import NaiveBayesClassifier
len(bagsofwords)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.25,random_state=123)


In [ ]:
x_lda=lda.fit(x_train,y_train).transform(x_train)
x_lda

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score


plt.figure(figsize=(35,20))
plt.scatter(x_lda[:,0],x_lda[:,1],c='Blue',s=200)

In [ ]:
naive = MultinomialNB()
classifier = naive.fit(x_train,y_train)
predict = classifier.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,predict)
cm

In [ ]:
accuracy = cm.trace()/cm.sum()
print(accuracy)
print(f1_score(y_test,predict))
print(precision_score(y_test,predict))
print(recall_score(y_test,predict))
print(roc_auc_score(y_test,predict))

In [ ]:
BernNB = BernoulliNB(binarize = 0.095)
BernNB.fit(x_train,y_train)
print(BernNB)

In [ ]:
y_pred_B = BernNB.predict(x_test)
print (accuracy_score(y_test,y_pred_B))
print (f1_score(y_test,y_pred_B))
print(precision_score(y_test,y_pred_B))
print(recall_score(y_test,y_pred_B))
print(roc_auc_score(y_test,y_pred_B))
cm = confusion_matrix(y_test,y_pred_B)
cm

In [ ]:
from sklearn import svm
from sklearn.svm import LinearSVC
clf = svm.LinearSVC()
clf.fit(x_train, y_train)  

In [ ]:
y_pred_SVM=clf.predict(x_test)
#y_pred_SVM
print (accuracy_score(y_test,y_pred_SVM))
print (f1_score(y_test,y_pred_SVM))
print(precision_score(y_test,y_pred_SVM))
print(recall_score(y_test,y_pred_SVM))
print(roc_auc_score(y_test,y_pred_SVM))
plt.figure(figsize=(35,20))
plt.scatter(pca_2d[:,0],pca_2d[:,1],c="Blue",s=200)

In [ ]:
from sklearn import svm
from sklearn.svm import LinearSVC
clf = svm.LinearSVC()
clf.fit(matrix, y)
y_pred_SVM2=clf.predict([])

In [ ]:
cm = confusion_matrix(y_test,y_pred_SVM)
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit(x_train)
pca_2d = pca.fit(x_train).transform(x_train)
plt.figure(figsize=(35,20))
plt.scatter(pca_2d[:,0],pca_2d[:,1],c="Blue",s=200)


In [ ]:
test_img=pca.transform(x_test)
from sklearn.linear_model import LogisticRegression

In [ ]:
lor=LogisticRegression(solver='lbfgs')

In [ ]:
lor.fit(x_train,y_train)

In [ ]:
ospdo=lor.predict(x_test)
print (accuracy_score(y_test,ospdo))
print (f1_score(y_test,ospdo))
print(precision_score(y_test,ospdo))
print(recall_score(y_test,ospdo))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
probs=lor.predict(x_test)
auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, probs)
# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
# show the plot
pyplot.show()

In [ ]:
oofd

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
model = KNeighborsClassifier(n_neighbors=3)
model.fit(x_train, y_train)
# predict probabilities
probs = model.predict_proba(x_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, probs)
# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
# show the plot
pyplot.show()